# Análise das Perguntas SQL com Python

## Localização de chamados do 1746
#### Utilize a tabela de Chamados do 1746 e a tabela de Bairros do Rio de Janeiro para as perguntas de 1-5.

1. Quantos chamados foram abertos no dia 01/04/2023?
2. Qual o tipo de chamado que teve mais teve chamados abertos no dia 01/04/2023?
3. Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?
4. Qual o nome da subprefeitura com mais chamados abertos nesse dia?
5. Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?


## Chamados do 1746 em grandes eventos
#### Utilize a tabela de Chamados do 1746 e a tabela de Ocupação Hoteleira em Grandes Eventos no Rio para as perguntas de 6-10. Para todas as perguntas considere o subtipo de chamado "Perturbação do sossego".

6. Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?
7. Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).
8. Quantos chamados desse subtipo foram abertos em cada evento?
9. Qual evento teve a maior média diária de chamados abertos desse subtipo?
10. Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023.

##### Importante: a tabela de Chamados do 1746 possui mais de 10M de linhas. Evite fazer consultas exploratórias na tabela sem um filtro ou limite de linhas para economizar sua cota no BigQuery!

## Imports

In [1]:
import pandas as pd

# Dados

In [2]:
ocup = pd.read_csv('datasets/ocupacao.csv')
bairro = pd.read_csv('datasets/bairro.csv')
chamado = pd.read_parquet('datasets/chamado.parquet')


In [3]:
ocup

,ano,data_inicial,data_final,evento,taxa_ocupacao
0,02/09 a 04/09 de 2022,2022-09-02,2022-09-04,Rock in Rio,0.8184
1,08/09 a 11/09 de 2022,2022-09-08,2022-09-11,Rock in Rio,0.9451
2,18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554
3,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251


## Trativas

In [7]:
# Para fazer o merge dos dataframes bairro + chamado é preciso converter o tipo da coluna id_bairro
bairro['id_bairro'] = bairro['id_bairro'].apply(lambda x: str(x)) # converter o id_bairro de bairro para str porque possui menos registros no dataframe

# Merge(join) de chamado + bairro
cham_bair = pd.merge(chamado, bairro, how='left', on='id_bairro')

# Coluna de data
cham_bair['data_inicio'] = pd.to_datetime(cham_bair['data_inicio'])
cham_bair['data_inicio'] = cham_bair['data_inicio'].apply(lambda x: x.strftime('%Y-%m-%d'))

# Merge cham_bair + ocup
# Função para gerar a lista de datas
def gerar_lista_datas(row):
    return pd.date_range(start=row['data_inicial'], end=row['data_final']).strftime('%Y-%m-%d').tolist() # essa função analisa as datas de inicio e fim e retorna uma lista com todas as datas do evento

# Aplicando a função em cada linha do DataFrame
ocup['periodo'] = ocup.apply(gerar_lista_datas, axis=1) # agora o dataframe ocup possui uma coluna com listas dos dias de cada evento

# Transformando o dataframe para que cada data da coluna 'periodo' seja uma linha
df_explodido = ocup.explode('periodo') # A função explode transforma listas em várias linhas, mantendo as demais colunas
df_explodido = df_explodido[['periodo', 'evento', 'taxa_ocupacao']] # Filtra o dataframe
df_explodido.columns = ['data_inicio', 'evento', 'taxa_ocupacao'] # muda o nome da coluna para ser possível fazer o merge

# Faz o join do dataframe cham_bair com o df_explodido
merged = pd.merge(cham_bair, df_explodido, how='left', on='data_inicio') 

In [9]:
df_explodido

,data_inicio,evento,taxa_ocupacao
0,2022-09-02,Rock in Rio,0.8184
0,2022-09-03,Rock in Rio,0.8184
0,2022-09-04,Rock in Rio,0.8184
1,2022-09-08,Rock in Rio,0.9451
1,2022-09-09,Rock in Rio,0.9451
1,2022-09-10,Rock in Rio,0.9451
1,2022-09-11,Rock in Rio,0.9451
2,2023-02-18,Carnaval,0.9554
2,2023-02-19,Carnaval,0.9554
2,2023-02-20,Carnaval,0.9554


In [7]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1653177 entries, 0 to 1653176
Data columns (total 26 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   id_chamado                   1653177 non-null  object 
 1   data_inicio                  1653177 non-null  object 
 2   id_bairro                    1580338 non-null  object 
 3   nome_unidade_organizacional  1653177 non-null  object 
 4   categoria                    1653177 non-null  object 
 5   tipo                         1653177 non-null  object 
 6   subtipo                      1653177 non-null  object 
 7   status                       1653177 non-null  object 
 8   longitude                    1206084 non-null  float64
 9   latitude                     1206084 non-null  float64
 10  situacao                     1653177 non-null  object 
 11  tipo_situacao                1653177 non-null  object 
 12  reclamacoes                  1653177 non-n

In [8]:
cham_bair.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1634336 entries, 0 to 1634335
Data columns (total 24 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   id_chamado                   1634336 non-null  object 
 1   data_inicio                  1634336 non-null  object 
 2   id_bairro                    1562182 non-null  object 
 3   nome_unidade_organizacional  1634336 non-null  object 
 4   categoria                    1634336 non-null  object 
 5   tipo                         1634336 non-null  object 
 6   subtipo                      1634336 non-null  object 
 7   status                       1634336 non-null  object 
 8   longitude                    1192576 non-null  float64
 9   latitude                     1192576 non-null  float64
 10  situacao                     1634336 non-null  object 
 11  tipo_situacao                1634336 non-null  object 
 12  reclamacoes                  1634336 non-n

## Respostas

1. **Quantos chamados foram abertos no dia 01/04/2023?**

In [10]:
abril_01 = cham_bair[cham_bair['data_inicio'] == '2023-04-01']
len(abril_01)

1756

2. **Qual o tipo de chamado que teve mais teve chamados abertos no dia 01/04/2023?**

In [11]:
abril_01.groupby('tipo').agg({'id_chamado':'count'}).sort_values('id_chamado', ascending=False).head(1)


,id_chamado
tipo,
Estacionamento irregular,366


3. **Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?**

In [12]:
abril_01.groupby('nome').agg({'id_chamado':'count'}).sort_values('id_chamado', ascending=False).head(3)


,id_chamado
nome,
Campo Grande,113
Tijuca,89
Barra da Tijuca,59


4. **Qual o nome da subprefeitura com mais chamados abertos nesse dia?**

In [13]:
abril_01.groupby('subprefeitura').agg({'id_chamado':'count'}).sort_values('id_chamado', ascending=False).head(3)

,id_chamado
subprefeitura,
Zona Norte,510
Zona Oeste,238
Zona Sul,233


5. **Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?**

In [14]:
print(f'Existem {len(abril_01[abril_01['nome'].isnull()])} chamados abertos nesse dia que não foram associados a bairro nem subprefeituras')

Existem 73 chamados abertos nesse dia que não foram associados a bairro nem subprefeituras


Isso acontece pois não há informações sobre o local no registro do chamado. Não há id do bairro, nem coordenadas e nem qualquer informação que possibilite identificar de onde vem o chamado.
<br>Levando em consideração que 01 de abril é um dia popularmente conhecido como "dia da mentira", esses registros podem ter ocorrido por conta de trotes e os indivíduos que efetuaram a ligação não quiseram informar nenhuma informação sobre o local referente ao chamado.
<br>Outra possibilidade seria erro de tabulação do atendente no momento de registrar o chamado, ou algum outro erro sistêmico.


6. **Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?**

In [15]:
# O dataframe já está previamente filtrado com o período do dia 01/01/202 e 31/12/2023(considerando que as extremidades sejam o primeiro e último dia deste período)
print(f'Foram abertos {sum(cham_bair['subtipo'] == 'Perturbação do sossego')} chamados com o subtipo "Perturbação do sossego" dentro deste período.')

Foram abertos 42830 chamados com o subtipo "Perturbação do sossego" dentro deste período.


7. **Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).**

In [16]:
eventos = merged[merged['evento'].isin(['Reveillon', 'Carnaval','Rock in Rio'])]
eventos[eventos['subtipo'] == 'Perturbação do sossego']

,id_chamado,data_inicio,id_bairro,nome_unidade_organizacional,categoria,tipo,subtipo,status,longitude,latitude,...,nome_regiao_planejamento,id_regiao_administrativa,nome_regiao_administrativa,subprefeitura,area,perimetro,geometry_wkt,geometry,evento,taxa_ocupacao
10610,18077088,2022-12-30,42,GM-RIO - Guarda Municipal do Rio de Janeiro,Serviço,Perturbação do sossego,Perturbação do sossego,Não constatado,-43.267038,-22.840262,...,Ramos,10.0,Ramos,Zona Norte,3.689833e+06,12231.009936,POLYGON ((-43.256280603077975 -22.837866685590...,"POLYGON((-43.256280603078 -22.8378666855908, -...",Reveillon,0.9251
14622,18079544,2022-12-31,128,GM-RIO - Guarda Municipal do Rio de Janeiro,Serviço,Perturbação do sossego,Perturbação do sossego,Sem possibilidade de atendimento,-43.300530,-23.009047,...,Barra da Tijuca,24.0,Barra Da Tijuca,Barra da Tijuca,4.815063e+07,47771.859388,MULTIPOLYGON (((-43.40294914282091 -22.9727164...,MULTIPOLYGON(((-43.3039867221538 -23.036655138...,Reveillon,0.9251
14659,18078456,2022-12-31,83,GM-RIO - Guarda Municipal do Rio de Janeiro,Serviço,Perturbação do sossego,Perturbação do sossego,Sem possibilidade de atendimento,NaN,NaN,...,Madureira,15.0,Madureira,Zona Norte,3.787613e+06,9574.374845,POLYGON ((-43.32430021516191 -22.8646410544458...,"POLYGON((-43.3243002151619 -22.8646410544459, ...",Reveillon,0.9251
14660,18078548,2022-12-31,49,GM-RIO - Guarda Municipal do Rio de Janeiro,Serviço,Perturbação do sossego,Perturbação do sossego,Sem possibilidade de atendimento,NaN,NaN,...,Penha,31.0,Vigario Geral,Zona Norte,1.973968e+06,6129.785809,POLYGON ((-43.320925265522085 -22.801617150146...,"POLYGON((-43.3209252655221 -22.8016171501469, ...",Reveillon,0.9251
14664,18077762,2022-12-30,83,GM-RIO - Guarda Municipal do Rio de Janeiro,Serviço,Perturbação do sossego,Perturbação do sossego,Sem possibilidade de atendimento,NaN,NaN,...,Madureira,15.0,Madureira,Zona Norte,3.787613e+06,9574.374845,POLYGON ((-43.32430021516191 -22.8646410544458...,"POLYGON((-43.3243002151619 -22.8646410544459, ...",Reveillon,0.9251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1633534,17685619,2022-09-11,144,GM-RIO - Guarda Municipal do Rio de Janeiro,Serviço,Perturbação do sossego,Perturbação do sossego,Não constatado,-43.548471,-22.910828,...,Campo Grande,18.0,Campo Grande,Zona Oeste,1.044451e+08,75083.751962,POLYGON ((-43.51515139864348 -22.8522866123935...,"POLYGON((-43.5151513986435 -22.8522866123935, ...",Rock in Rio,0.9451
1633543,17684441,2022-09-11,161,GM-RIO - Guarda Municipal do Rio de Janeiro,Serviço,Perturbação do sossego,Perturbação do sossego,Não constatado,NaN,NaN,...,Centro,2.0,Centro,Centro,2.983258e+05,3849.181818,POLYGON ((-43.18166120770202 -22.9120798224676...,"POLYGON((-43.181661207702 -22.9120798224677, -...",Rock in Rio,0.9451
1633546,17683933,2022-09-10,132,GM-RIO - Guarda Municipal do Rio de Janeiro,Serviço,Perturbação do sossego,Perturbação do sossego,Não constatado,NaN,NaN,...,Barra da Tijuca,24.0,Barra Da Tijuca,Barra da Tijuca,3.065562e+07,38561.310855,POLYGON ((-43.437913649227745 -22.995137363384...,"POLYGON((-43.4379136492277 -22.9951373633849, ...",Rock in Rio,0.9451
1633549,17664435,2022-09-04,146,GM-RIO - Guarda Municipal do Rio de Janeiro,Serviço,Perturbação do sossego,Perturbação do sossego,Não constatado,NaN,NaN,...,Campo Grande,18.0,Campo Grande,Zona Oeste,8.287883e+06,15356.885005,POLYGON ((-43.603809592060216 -22.933967544280...,"POLYGON((-43.6038095920602 -22.9339675442804, ...",Rock in Rio,0.8184


8. **Quantos chamados desse subtipo foram abertos em cada evento?**

In [17]:
eventos[eventos['subtipo'] == 'Perturbação do sossego'].groupby('evento').agg({'id_chamado':'count'})

,id_chamado
evento,
Carnaval,241
Reveillon,139
Rock in Rio,834


9. **Qual evento teve a maior média diária de chamados abertos desse subtipo?**

In [18]:
eventos_2 = eventos[eventos['subtipo'] == 'Perturbação do sossego'].groupby(['evento','data_inicio']).agg({'id_chamado':'count'}).reset_index()
eventos_2.groupby('evento').agg({'id_chamado':'mean'})

,id_chamado
evento,
Carnaval,60.250000
Reveillon,46.333333
Rock in Rio,119.142857


O Rock in Rio teve uma média diária de chamados abertos do subtipo Perturbação ao sossego bem maior do que os outros eventos

10. **Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023.**

In [19]:
df2 = cham_bair[cham_bair['subtipo'] == 'Perturbação do sossego'].groupby('data_inicio').agg({'id_chamado':'count'})
df2.mean()

id_chamado    61.982634
dtype: float64

O Rock em Rio definitivamente possui uma média de chamados para Perturbação ao sossego bem maior do que a média geral(quase o dobro).
<br>Sendo uma média diária de 119 chamados deste subtipo só no Rock in Rio contra uma média de 62 chamados diários ao longo dos anos 2022 e 2023 juntos.
<br> O Carnaval possui uma média de chamados diários muito próxima do normal, enquanto é possível notar uma queda nessa estimativa durante o Reveillon. 